In [ ]:
# default_exp series.find

# series.find
> Tools for taking a given filepath and searching for a specific type of MR series within the DICOM files on that path

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from dicomtools.basics import *
from dicomtools.series.predict import *

In [ ]:
#export
model_path = Path('dicomtools/models/mr-brain-series-select-rf.skl')

_planes = ['ax', 'cor', 'sag']
_y_names = pd.Index([
    't1', 
    't2',
    'swi', 
    'dwi', 
    'other', 
    'flair', 
    'adc', 
    'loc', 
    'spgr',
    'mra'
])

_features = ['MRAcquisitionType', 'AngioFlag', 'SliceThickness', 'RepetitionTime',
       'EchoTime', 'EchoTrainLength', 'PixelSpacing', 'ContrastBolusAgent',
       'InversionTime', 'DiffusionBValue', 'seq_E', 'seq_EP', 'seq_G',
       'seq_GR', 'seq_I', 'seq_IR', 'seq_M', 'seq_P', 'seq_R', 'seq_S',
       'seq_SE', 'var_E', 'var_K', 'var_MP', 'var_MTC', 'var_N', 'var_O',
       'var_OSP', 'var_P', 'var_S', 'var_SK', 'var_SP', 'var_SS', 'var_TOF',
       'opt_1', 'opt_2', 'opt_A', 'opt_ACC_GEMS', 'opt_B', 'opt_C', 'opt_D',
       'opt_E', 'opt_EDR_GEMS', 'opt_EPI_GEMS', 'opt_F', 'opt_FAST_GEMS',
       'opt_FC', 'opt_FC_FREQ_AX_GEMS', 'opt_FC_SLICE_AX_GEMS',
       'opt_FILTERED_GEMS', 'opt_FR_GEMS', 'opt_FS', 'opt_FSA_GEMS',
       'opt_FSI_GEMS', 'opt_FSL_GEMS', 'opt_FSP_GEMS', 'opt_FSS_GEMS', 'opt_G',
       'opt_I', 'opt_IFLOW_GEMS', 'opt_IR', 'opt_IR_GEMS', 'opt_L', 'opt_M',
       'opt_MP_GEMS', 'opt_MT', 'opt_MT_GEMS', 'opt_NPW', 'opt_P', 'opt_PFF',
       'opt_PFP', 'opt_PROP_GEMS', 'opt_R', 'opt_RAMP_IS_GEMS', 'opt_S',
       'opt_SAT1', 'opt_SAT2', 'opt_SAT_GEMS', 'opt_SEQ_GEMS', 'opt_SP',
       'opt_T', 'opt_T2FLAIR_GEMS', 'opt_TRF_GEMS', 'opt_VASCTOF_GEMS',
       'opt_VB_GEMS', 'opt_W', 'opt_X', 'opt__']

_output_columns = [
    'fname',
    # Patient info
    'PatientID',
    # Study info
    'StudyInstanceUID',
    'StudyID',
    # Series info
    'SeriesInstanceUID',
    'SeriesNumber',
    'SeriesDescription',
    'AcquisitionNumber',
    # Image info and features
    'InstanceNumber',
    # extracted values
    'plane',
    'contrast'
]


In [ ]:
#export
class Finder():
    "A class for finding DICOM files of a specified sequence type from a specific ."
    def __init__(self, path):
        self.root = path
        self.fns, self.dicoms = get_dicoms(self.root)
        self.dicoms = preprocess(self.dicoms)
        self.labels = extract_labels(self.dicoms)
        self.dicoms = self.dicoms.join(self.labels[['plane', 'contrast']])
        
    def predict(self,  model_path=model_path, features=_features, ynames=_y_names, **kwargs):
        try:
            self.clf = load(model_path)
        except FileNotFoundError as e:
            print("No model found. Try again by passing the `model_path` keyword argument.")
            raise
        self.features = features
        self.ynames = ynames
        preds = self.clf.predict(self.dicoms[features])
        self.preds = ynames.take(preds)
        self.probas = self.clf.predict_proba(self.dicoms[features])
        
    def find(self, plane='ax', seq='t1', contrast=True, thresh=0.8, **kwargs):
        try:
            self.probas
        except AttributeError:
            print("Prediction not yet performed. Please run `Finder.predict()` and try again.")
            raise
        preds = np.argwhere(self.probas > 0.8)
        ind = preds[:, 0]
        pred_names = _y_names.take(preds[:, 1])
        df = pd.DataFrame(pred_names, index=ind, columns=['seq_pred'])
        df = self.dicoms[_output_columns].join(df)
        return df.query(f'plane == "{plane}" and seq_pred == "{seq}" and contrast == {int(contrast)}')
    

In [ ]:
#export
add_docs(Finder,
         predict="Obtains predictions from the model specified in `model_path`",
         find="Returns a `pandas.DataFrame` with predicted sequences matching the query at the specified threshold")